# LangChain 核心模块 Agent - ReAct

ReAct 核心思想是 推理+操作，本示例以` Google Search` 和 `LLM Math` 作为可选操作集合（toolkits），实现 ReAct 功能。

以下`SERPAPI_API_KEY`仅为示例，请访问 https://serpapi.com 注册账号并替换为自己的 `API_KEY`（每月100次免费调用）

In [2]:
import os

# 更换为自己的 Serp API KEY
os.environ["SERPAPI_API_KEY"] = "1e3c93e0753ac224098370cd71da86150c6609caba6b5aaa307e04b72a5006e1"

In [3]:
from langchain_openai import OpenAI

llm = OpenAI(temperature=0)

In [4]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType

#加载 LangChain 内置的 Tools
tools = load_tools(["serpapi", "llm-math"], llm=llm)
print(tools)

[Tool(name='Search', description='A search engine. Useful for when you need to answer questions about current events. Input should be a search query.', func=<bound method SerpAPIWrapper.run of SerpAPIWrapper(search_engine=<class 'serpapi.google_search.GoogleSearch'>, params={'engine': 'google', 'google_domain': 'google.com', 'gl': 'us', 'hl': 'en'}, serpapi_api_key='a17b7002a31071ddde277a78ee1dc6848b272676d0d7ef2813bc265ef3945767', aiosession=None)>, coroutine=<bound method SerpAPIWrapper.arun of SerpAPIWrapper(search_engine=<class 'serpapi.google_search.GoogleSearch'>, params={'engine': 'google', 'google_domain': 'google.com', 'gl': 'us', 'hl': 'en'}, serpapi_api_key='a17b7002a31071ddde277a78ee1dc6848b272676d0d7ef2813bc265ef3945767', aiosession=None)>), Tool(name='Calculator', description='Useful for when you need to answer questions about math.', func=<bound method Chain.run of LLMMathChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['question'], template='Translate a m

### 使用语言模型（GPT-3）

In [8]:
# from langchain import hub

# # Get the prompt to use - you can modify this!
# prompt = hub.pull("hwchase17/openai-functions-agent")
# prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [7]:
# # 实例化 ZERO_SHOT_REACT Agent
# from langchain.agents import AgentExecutor
# from langchain.agents import create_react_agent
# # initialize agent with tools
# agent = create_react_agent(llm, tools, prompt)
# agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [10]:
agent = initialize_agent(
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    tools=tools,
    llm=llm,
    verbose=True,
    handle_parsing_errors=True
)

/home/jerry/miniconda3/envs/aienv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [11]:
agent.run("谁是莱昂纳多·迪卡普里奥的女朋友？她现在年龄的0.43次方是多少?")

/home/jerry/miniconda3/envs/aienv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
 I should use a search engine to find information about Leonardo DiCaprio's girlfriend and then use a calculator to calculate her age.
Action: Search
Action Input: "Leonardo DiCaprio girlfriend"
Observation: Model Vittoria Ceretti, who is dating Leonardo DiCaprio, landed an Elle cover story to talk about modeling and not talk about Leo.
Thought: Now I need to use a calculator to calculate her age.
Action: Calculator
Action Input: 0.43^
Observation: Answer: 0.18489999999999998
Thought: I now know the final answer.
Final Answer: Vittoria Ceretti's age is approximately 0.1849.

> Finished chain.


"Vittoria Ceretti's age is approximately 0.1849."

### 使用聊天模型（GPT-4）

In [12]:
from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI(model="gpt-4-1106-preview", temperature=0)
agent = initialize_agent(tools, chat_model, agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
agent.run("谁是莱昂纳多·迪卡普里奥的女朋友？她现在年龄的0.43次方是多少?")



> Entering new AgentExecutor chain...
Question: 谁是莱昂纳多·迪卡普里奥的女朋友？
Thought: 首先，我需要找出莱昂纳多·迪卡普里奥当前的女朋友是谁。这是一个关于当前事件的问题，所以我将使用搜索工具来找到答案。
Action:
```
{
  "action": "Search",
  "action_input": "Leonardo DiCaprio current girlfriend 2023"
}
```

Observation: Vittoria Ceretti
Thought:Now that I know Leonardo DiCaprio's current girlfriend is Vittoria Ceretti, I need to find out her age to calculate the 0.43rd power of her age.

Thought: I will use the search tool to find out Vittoria Ceretti's age.

Action:
```
{
  "action": "Search",
  "action_input": "Vittoria Ceretti age"
}
```
Observation: 25 years
Thought:Now that I know Vittoria Ceretti is 25 years old, I can calculate the 0.43rd power of her age.

Action:
```
{
  "action": "Calculator",
  "action_input": {
    "expression": "25^0.43"
  }
}
```

Observation: Answer: 3.991298452658078
Thought:I now know the final answer to the second part of the question.

Final Answer: The 0.43rd power of Vittoria Ceretti's age, who is 25 years old, is a

"The 0.43rd power of Vittoria Ceretti's age, who is 25 years old, is approximately 3.991."

In [17]:
tools[0].name="Intermediate Answer"
tools = [tools[0]]
agent = initialize_agent(
    agent=AgentType.SELF_ASK_WITH_SEARCH ,
    tools=tools,
    llm=OpenAI(temperature=0),
    verbose=True
)
agent.run("谁是莱昂纳多·迪卡普里奥的女朋友？她现在年龄的0.43次方是多少?")



> Entering new AgentExecutor chain...
 Yes.
Follow up: 莱昂纳多·迪卡普里奥的女朋友是谁？
Intermediate answer: ['11月1日，有媒体拍到莱昂纳多与新任女友，25岁的意大利超模维多利亚·切蕾蒂（Vittoria Ceretti）在某夜总会亲密拥抱热吻的画面，行为十分豪放。 打开网易 ...', '8月31日，据外媒报道，47岁的莱昂纳多·迪卡普里奥与刚满25岁不久的模特女友卡米拉·莫罗在今年刚刚入夏的时候，以“顺其自然”的方式分手，5年感情画上 ...', '11月1日，有媒体拍到莱昂纳多与新任女友，25岁的意大利超模维多利亚·切蕾蒂（Vittoria Ceretti）在某夜总会亲密拥抱热吻的画面，行为十分豪放。 图片. 两 ...', '19岁的模特伊甸园·波拉尼被认为是“泰坦尼克号”才子莱昂纳多·迪卡普里奥的新女友。 《每日邮报》刊登了一张演员和年轻女友的照片。', '刚和25岁超模女友卡米拉分手的小李子莱昂纳多·迪卡普里奥，被曝22岁新女友“火速上线”。 这位被换女友如换衣服的小李子“临幸”的妙龄女子就是乌克兰超模玛莉 ...', '今年8月，莱昂纳多与交往四年多的女友卡米拉·莫罗内（Camila Morrone）分手，之后他多次被拍到与27岁的超模吉吉·哈迪德（Gigi Hadid）在一起。9月，有消息 ...', '梅根罗氏她也是很好的朋友梁咏琪哈迪德，最著名的前火焰之一莱昂纳多·迪卡普里奥，这位演员最近几天也会和他一起吃饭伦敦在英国首都的一家著名餐厅。 和 ...', '根据著名八卦网站统计，小李子已经交往了61任女友（包含现任），其中未官宣的“绯闻女友”就有22个。 可谓是“铁打的小李子，流水的超模”。', '... 莱昂纳多·迪卡普里奥的女友却能永远保持年轻。近日，外媒曝光了小李子的新女友，是25岁超模Vittoria Ceretti，两人被拍到在夜店拥吻。自此也再一次 ...', '去年12月下旬，莱昂纳多·迪卡普里奥（Leonardo DiCaprio）被拍到和女友维多利亚·拉马斯（Victoria Lamas）在游艇欢度圣诞假期。 场面欢悦，又是秧歌又 ...']
Follow up: 莱昂纳多·迪卡普里奥的女朋友现在多大了？
Intermediate answer

ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse output: 